In [ ]:
pip install google_streetview.api

ERROR: Could not find a version that satisfies the requirement google_streetview.api (from versions: none)
ERROR: No matching distribution found for google_streetview.api


In [ ]:
## The following code can load the trained model to do homeless detection for a given area of location.
## This will save the predicted(and a csv file including details about each image) and original image in a folder

In [ ]:
import google_streetview.api
import os
import time
import requests
import torch
import torchvision.transforms.functional as TF
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw, ImageFont
import csv

# Google Street View API key
#################################################################################################### Need to update the Google API key below
API_KEY = ""  # Replace with your actual API key
####################################################################################################


# Define bounding box coordinates (area to cover)
top_left = (34.044133, -118.243896)
bottom_right = (34.038049, -118.242965)

# Grid dimensions
num_rows = 5   # vertical (latitude)
num_cols = 10  # horizontal (longitude)

# Output directories
#################################################################################################### Needs to update your location where to download the images at
download_dir = "Google Street View Downloader/Test3"
original_dir = "Google Street View Downloader/Original possible homeless GSV images3"
predicted_dir = "Google Street View Downloader/Predicted homeless GSV3"
csv_path = os.path.join(predicted_dir, "predictions.csv")

# Create directories if they don't exist
os.makedirs(download_dir, exist_ok=True)
os.makedirs(original_dir, exist_ok=True)
os.makedirs(predicted_dir, exist_ok=True)

# Load the trained Fast-RCNN model
num_classes = 5  # Background + 4 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn(weights=None, num_classes=num_classes)
model.load_state_dict(torch.load("model_final_2.pth", map_location=device))
model.to(device)
model.eval()

# Class mapping
label_map = {
    1: "Homeless_People",
    2: "Homeless_Encampments",
    3: "Homeless_Cart",
    4: "Homeless_Bike"
}

# Generate evenly spaced grid points
latitudes = [top_left[0] + i * (bottom_right[0] - top_left[0]) / (num_rows - 1) for i in range(num_rows)]
longitudes = [top_left[1] + j * (bottom_right[1] - top_left[1]) / (num_cols - 1) for j in range(num_cols)]
grid_points = [(lat, lon) for lat in latitudes for lon in longitudes]

# Draw bounding boxes
def draw_predictions(image, boxes, labels, scores, score_thresh=0.5):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for i, box in enumerate(boxes):
        if scores[i] < score_thresh:
            continue
        xmin, ymin, xmax, ymax = box
        cls_id = int(labels[i])
        cls_name = label_map.get(cls_id, str(cls_id))
        draw.rectangle([xmin, ymin, xmax, ymax], outline=(255, 0, 0), width=3)
        draw.text((xmin, ymin - 10), f"{cls_name} {scores[i]:.2f}", fill=(255, 0, 0), font=font)
    return image

# CSV headers
csv_fields = ["filename", "lat", "lon", "heading", "date", "class", "confidence"]
if not os.path.exists(csv_path):
    with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=csv_fields)
        writer.writeheader()

# Run detection and logging
for idx, (lat, lon) in enumerate(grid_points):
    for heading in [0, 180]:
        params = [{
            'size': '640x640',
            'location': f'{lat},{lon}',
            'heading': str(heading),
            'pitch': '0',
            'key': API_KEY
        }]

        results = google_streetview.api.results(params)

        try:
            metadata = results.metadata[0]
            pano_id = metadata.get('pano_id', 'unknown')
            date = metadata.get('date', 'unknown')
            filename = f"streetview_{pano_id}_{date}_{lat}_{lon}_heading{heading}.jpg"
            original_path = os.path.join(original_dir, filename)
            predicted_path = os.path.join(predicted_dir, filename)

            if results.links:
                response = requests.get(results.links[0])
                with open(original_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded: {filename}")

                image = Image.open(original_path).convert("RGB")
                img_tensor = TF.to_tensor(image).unsqueeze(0).to(device)

                with torch.no_grad():
                    prediction = model(img_tensor)[0]

                pred_boxes = prediction["boxes"].cpu().numpy()
                pred_labels = prediction["labels"].cpu().numpy()
                pred_scores = prediction["scores"].cpu().numpy()

                if len(pred_boxes) > 0:
                    print(f"✅ Homeless entities detected in {filename}!")
                    image.save(original_path)
                    pred_image = draw_predictions(image.copy(), pred_boxes, pred_labels, pred_scores)
                    pred_image.save(predicted_path)

                    with open(csv_path, mode='a', newline='', encoding='utf-8') as f:
                        writer = csv.DictWriter(f, fieldnames=csv_fields)
                        for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
                            if score < 0.5:
                                continue
                            cls_name = label_map.get(int(label), str(label))
                            writer.writerow({
                                "filename": filename,
                                "lat": lat,
                                "lon": lon,
                                "heading": heading,
                                "date": date,
                                "class": cls_name,
                                "confidence": round(float(score), 3)
                            })
                else:
                    print(f"❌ No homeless-related objects detected in {filename}.")

            else:
                print(f"No image available for {lat}, {lon}, heading {heading}")

            time.sleep(1)

        except Exception as e:
            print(f"Error processing {lat}, {lon}, heading {heading}: {e}")

print("Processing completed.")

ModuleNotFoundError: No module named 'google_streetview'

In [ ]:
import os
import base64
import re
import pandas as pd
from ipyleaflet import Map, Marker, Popup, MarkerCluster
from ipywidgets import HTML
from IPython.display import display

# Paths
#################################################################################################### This is the folder where your predicted images located at
image_dir = "Google Street View Downloader/Predicted homeless GSV3"
csv_path = os.path.join(image_dir, "predictions.csv")
####################################################################################################


# Load CSV
df = pd.read_csv(csv_path)

# Group detections by filename
grouped = df.groupby("filename")

# Extract info from filename
location_groups = {}
for filename, group in grouped:
    match = re.search(r'streetview_.*?_(\d{4}-\d{2})_(-?\d+\.\d+)_(-?\d+\.\d+)_heading(\d+)', filename)
    if not match:
        continue
    date, lat_str, lon_str, heading = match.groups()
    lat = float(lat_str)
    lon = float(lon_str)
    location_groups[filename] = {
        "filename": filename,
        "date": date,
        "lat": lat,
        "lon": lon,
        "heading": heading,
        "detections": group.to_dict("records")
    }

# Initialize map
if location_groups:
    first = next(iter(location_groups.values()))
    m = Map(center=(first['lat'], first['lon']), zoom=18, scroll_wheel_zoom=True, layout=dict(width="1400px", height="1000px"))
    cluster = MarkerCluster()

    for key, image_info in location_groups.items():
        filename = image_info["filename"]
        image_path = os.path.join(image_dir, filename)

        if not os.path.exists(image_path):
            continue

        with open(image_path, "rb") as f:
            img_base64 = base64.b64encode(f.read()).decode("utf-8")

        # Count predictions by class
        class_counts = {}
        for det in image_info["detections"]:
            cls = det["class"]
            class_counts[cls] = class_counts.get(cls, 0) + 1

        pred_lines = "<br>".join([f"{cls} × {count}" for cls, count in class_counts.items()])
        pred_summary = f"<b>Predictions:</b> {sum(class_counts.values())} object(s)<br>{pred_lines}<br>"

        # Define image width
        image_display_width = 600
        popup_height = 800

        # HTML block with zoom and drag
        html_content = f"""
        <div style="width: {image_display_width + 50}px; height: {popup_height}px; overflow-y: auto; font-size: 16px;">
            <b>Date:</b> {image_info['date']}<br>
            <b>Location:</b> {image_info['lat']:.6f}, {image_info['lon']:.6f}<br>
            <b>Heading:</b> {image_info['heading']}°<br>
            {pred_summary}

            <div style="width: {image_display_width}px; overflow: hidden; border: 1px solid #ccc;">
                <div style="display: inline-block; transform: scale(1); transform-origin: center center; cursor: grab;"
                     onmousedown="this.style.cursor='grabbing'; this.dataset.drag='1'; this.dataset.x=event.clientX; this.dataset.y=event.clientY;"
                     onmouseup="this.style.cursor='grab'; this.dataset.drag='0';"
                     onmouseleave="this.style.cursor='grab'; this.dataset.drag='0';"
                     onmousemove="if(this.dataset.drag==='1'){{ let dx=event.clientX-this.dataset.x; let dy=event.clientY-this.dataset.y; this.parentElement.scrollLeft -= dx; this.parentElement.scrollTop -= dy; this.dataset.x=event.clientX; this.dataset.y=event.clientY; }}"
                     onwheel="
                        event.preventDefault();
                        let scale = parseFloat(this.style.transform.match(/scale\\(([^)]+)\\)/)[1]);
                        scale += (event.deltaY < 0 ? 0.1 : -0.1);
                        scale = Math.min(Math.max(1, scale), 5);
                        const rect = this.getBoundingClientRect();
                        const offsetX = event.clientX - rect.left;
                        const offsetY = event.clientY - rect.top;
                        this.style.transformOrigin = `${{offsetX}}px ${{offsetY}}px`;
                        this.style.transform = `scale(${{scale}})`;
                     ">
                    <img src="data:image/jpeg;base64,{img_base64}" style="width: {image_display_width}px;">
                </div>
            </div>
        </div>
        """

        popup = Popup(
            location=(image_info["lat"], image_info["lon"]),
            child=HTML(value=html_content),
            close_button=True,
            auto_close=False,
            max_width=image_display_width + 60  # buffer for borders and scrollbar
        )

        marker = Marker(location=(image_info["lat"], image_info["lon"]))
        marker.popup = popup
        cluster.markers += (marker,)

    m.add_layer(cluster)
    display(m)
else:
    print("No valid locations found in CSV.")


Map(center=[34.044133, -118.2431718888889], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…